In [1]:
#Import rapids library,
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#import cudf
#import cupy as cp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/rapids/rapids.0.14.0
/kaggle/input/rapids/rapids.0.15.0


In [3]:
#cudf.__version__

In [4]:
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
#from cuml.linear_model import LogisticRegression

In [5]:
train_features=pd.read_csv('../input/lish-moa/train_features.csv')
test_features=pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
sample_submission=pd.read_csv('../input/lish-moa/sample_submission.csv')

In [6]:
train_features.head()

sig_id cp_type  cp_time cp_dose     g-0     g-1     g-2     g-3  \
0  id_000644bb2  trt_cp       24      D1  1.0620  0.5577 -0.2479 -0.6208   
1  id_000779bfc  trt_cp       72      D1  0.0743  0.4087  0.2991  0.0604   
2  id_000a6266a  trt_cp       48      D1  0.6280  0.5817  1.5540 -0.0764   
3  id_0015fd391  trt_cp       48      D1 -0.5138 -0.2491 -0.2656  0.5288   
4  id_001626bd3  trt_cp       72      D2 -0.3254 -0.4009  0.9700  0.6919   

      g-4     g-5  ...    c-90    c-91    c-92    c-93    c-94    c-95  \
0 -0.1944 -1.0120  ...  0.2862  0.2584  0.8076  0.5523 -0.1912  0.6584   
1  1.0190  0.5207  ... -0.4265  0.7543  0.4708  0.0230  0.2957  0.4899   
2 -0.0323  1.2390  ... -0.7250 -0.6297  0.6103  0.0223 -1.3240 -0.3174   
3  4.0620 -0.8095  ... -2.0990 -0.6441 -5.6300 -1.3780 -0.8632 -1.2880   
4  1.4180 -0.8244  ...  0.0042  0.0048  0.6670  1.0690  0.5523 -0.3031   

     c-96    c-97    c-98    c-99  
0 -0.3981  0.2139  0.3801  0.4176  
1  0.1522  0.1241  0.6077  0.7371  
2 -0.6417 -0.2187 -1.4080  0.6931  
3 -1.6210 -0.8784 -0.3876 -0.8154  
4  0.1094  0.2885 -0.3786  0.7125  

[5 rows x 876 columns]

In [7]:
targets=train_targets_scored.columns[1:]
len(targets)

206

In [8]:
##train features,
train_features=pd.concat([train_features,pd.get_dummies(train_features['cp_time'],prefix='cp_time')],axis=1)
train_features=pd.concat([train_features,pd.get_dummies(train_features['cp_type'],prefix='cp_type')],axis=1)
train_features=pd.concat([train_features,pd.get_dummies(train_features['cp_dose'],prefix='cp_dose')],axis=1)
train_features=train_features.drop(['cp_time','cp_type','cp_dose'],axis=1)

#test features,
test_features=pd.concat([test_features,pd.get_dummies(test_features['cp_time'],prefix='cp_time')],axis=1)
test_features=pd.concat([test_features,pd.get_dummies(test_features['cp_type'],prefix='cp_type')],axis=1)
test_features=pd.concat([test_features,pd.get_dummies(test_features['cp_dose'],prefix='cp_dose')],axis=1)
test_features=test_features.drop(['cp_time','cp_type','cp_dose'],axis=1)

In [9]:
kfolds=StratifiedKFold(n_splits=5,random_state=42,shuffle=True)

In [10]:
tar=['5-alpha_reductase_inhibitor','atp-sensitive_potassium_channel_antagonist','11-beta-hsd1_inhibitor']

In [11]:
feats=[c for c in train_features.columns if c not in ['sig_id']]

In [12]:
def log_loss_metric(y_true,y_preds):
    """
    Custom Log loss
    """
    y_preds_clipped=np.clip(y_preds,1e-15,1-1e-15)
    loss=np.mean(y_true*np.log(y_preds_clipped) + (1-y_true)*np.log(1-y_preds_clipped))
    return loss

In [13]:
scores=[]
oof_preds=np.zeros(train_features.shape[0])
sub_preds=np.zeros(test_features.shape[0])
feature_importance_df=pd.DataFrame()
non_pred_class=[]
score_dict=dict()
lgb_round_dict=defaultdict(int)
for class_name in targets:
    class_preds=np.zeros(len(train_targets_scored))
    train_targets=train_targets_scored[class_name]
    if train_targets.sum()>5:
        for fold_,(trn_,val_) in enumerate(kfolds.split(train_features,train_targets)):
            print(f'Started training fold {fold_} and class name {class_name}')
            trn_x,trn_y=train_features[feats].iloc[trn_],train_targets.iloc[trn_]
            val_x,val_y=train_features[feats].iloc[val_],train_targets.iloc[val_]
            #print(type(trn_x),type(trn_y),type(val_x),type(val_y))
            clf=lgb.LGBMClassifier(
            n_estimators=4000,
            learning_rate=0.03,
            num_leaves=30,
            colsample_bytree=.8,
            subsample=.9,
            max_depth=7,
            reg_alpha=.1,
            reg_lambda=.1,
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
            importance_type='split',
            device='gpu'
        )
        
            clf.fit(trn_x,trn_y,eval_set=[(trn_x,trn_y),(val_x,val_y)],early_stopping_rounds=50,
                         verbose=50)
            #print(model.best_iteration)
            class_preds[val_]=clf.predict_proba(val_x,num_iteration=clf.best_iteration_)[:,1]
            lgb_round_dict[class_name]+=clf.best_iteration_
            score=roc_auc_score(train_targets[val_],class_preds[val_])
            sub_preds+=clf.predict_proba(test_features[feats],num_iteration=clf.best_iteration_)[:,1]/5
            print(f'Fold {fold_} Class name {class_name} AUC {score} in {clf.best_iteration_} rounds')
            fold_importance_df=pd.DataFrame()
            fold_importance_df['features']=feats
            fold_importance_df['importance']=clf.feature_importances_
            fold_importance_df['class_names']=class_name
            fold_importance_df['folds']=fold_
            feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0) 
           # print(feature_importance_df.head())
            
        sample_submission[class_name]=sub_preds
        print(f'------Class {class_name} Full AUC Score  {roc_auc_score(train_targets,class_preds)}------')
        train_targets_scored[class_name+'_oof']=class_preds
        scores.append(roc_auc_score(train_targets,class_preds))
        score_dict[class_name]=roc_auc_score(train_targets,class_preds)
        
    else:
        non_pred_class.append(class_name)

    
print(f'Total CV Score {np.mean(scores)}')

Started training fold 0 and class name 5-alpha_reductase_inhibitor
Training until validation scores don't improve for 50 rounds
[50]	training's binary_logloss: 0.00328237	valid_1's binary_logloss: 0.00532572
Early stopping, best iteration is:
[3]	training's binary_logloss: 0.00588211	valid_1's binary_logloss: 0.00526151
Fold 0 Class name 5-alpha_reductase_inhibitor AUC 0.5705532212885154 in 3 rounds
Started training fold 1 and class name 5-alpha_reductase_inhibitor
Training until validation scores don't improve for 50 rounds
[50]	training's binary_logloss: 0.00335626	valid_1's binary_logloss: 0.00511958
[100]	training's binary_logloss: 0.00131699	valid_1's binary_logloss: 0.00501936
Early stopping, best iteration is:
[96]	training's binary_logloss: 0.00141873	valid_1's binary_logloss: 0.00501321
Fold 1 Class name 5-alpha_reductase_inhibitor AUC 0.7628851540616246 in 96 rounds
Started training fold 2 and class name 5-alpha_reductase_inhibitor
Training until validation scores don't impro

In [14]:
sample_submission.head()

sig_id  5-alpha_reductase_inhibitor  11-beta-hsd1_inhibitor  \
0  id_0004d9e33                     0.000458                0.001120   
1  id_001897cda                     0.000218                0.000932   
2  id_002429b5b                     0.000214                0.000810   
3  id_00276f245                     0.000276                0.000811   
4  id_0027f1083                     0.000480                0.001178   

   acat_inhibitor  acetylcholine_receptor_agonist  \
0        0.002109                        0.008193   
1        0.001908                        0.007317   
2        0.001796                        0.005942   
3        0.001786                        0.007502   
4        0.002163                        0.010231   

   acetylcholine_receptor_antagonist  acetylcholinesterase_inhibitor  \
0                           0.020366                        0.023298   
1                           0.014264                        0.017479   
2                           0.011533                        0.014131   
3                           0.016022                        0.018984   
4                           0.025535                        0.027825   

   adenosine_receptor_agonist  adenosine_receptor_antagonist  \
0                    0.024943                       0.029139   
1                    0.018834                       0.023750   
2                    0.016366                       0.019555   
3                    0.020514                       0.024184   
4                    0.029834                       0.032979   

   adenylyl_cyclase_activator  ...  tropomyosin_receptor_kinase_inhibitor  \
0                    0.029178  ...                               0.435903   
1                    0.023844  ...                               0.430400   
2                    0.019566  ...                               0.282484   
3                    0.024197  ...                               0.354488   
4                    0.033004  ...                               0.403779   

   trpv_agonist  trpv_antagonist  tubulin_inhibitor  \
0      0.436538         0.438527           0.439102   
1      0.430896         0.433148           0.433645   
2      0.282819         0.284993           0.285571   
3      0.354823         0.356765           0.357920   
4      0.404125         0.406281           0.407457   

   tyrosine_kinase_inhibitor  ubiquitin_specific_protease_inhibitor  \
0                   0.440301                               0.440495   
1                   0.445635                               0.445851   
2                   0.287364                               0.287547   
3                   0.359606                               0.359800   
4                   0.408821                               0.409031   

   vegfr_inhibitor  vitamin_b  vitamin_d_receptor_agonist  wnt_inhibitor  
0         0.441580   0.443411                    0.443826       0.444846  
1         0.448010   0.448741                    0.449386       0.450703  
2         0.288531   0.289624                    0.290236       0.291373  
3         0.360713   0.361717                    0.361822       0.363117  
4         0.410086   0.411664                    0.411765       0.412669  

[5 rows x 207 columns]

In [15]:
feature_importance_df.to_csv('feature_importance.csv',index=False)
sample_submission.to_csv('submission.csv',index=False)

In [16]:
score_dict

{'5-alpha_reductase_inhibitor': 0.830160252528124,
 '11-beta-hsd1_inhibitor': 0.6898790179488616,
 'acat_inhibitor': 0.44420975199663726,
 'acetylcholine_receptor_agonist': 0.6701940043131873,
 'acetylcholine_receptor_antagonist': 0.6552813860092664,
 'acetylcholinesterase_inhibitor': 0.5977091246690166,
 'adenosine_receptor_agonist': 0.6331517333832148,
 'adenosine_receptor_antagonist': 0.635331024959946,
 'adenylyl_cyclase_activator': 0.9982669523569447,
 'adrenergic_receptor_agonist': 0.7063918148525692,
 'adrenergic_receptor_antagonist': 0.6605507944629203,
 'akt_inhibitor': 0.8404848127561619,
 'aldehyde_dehydrogenase_inhibitor': 0.9965016291726925,
 'alk_inhibitor': 0.8841564305345224,
 'ampk_activator': 0.5695389743158839,
 'analgesic': 0.7063236982886593,
 'androgen_receptor_agonist': 0.6216803066004657,
 'androgen_receptor_antagonist': 0.6181586767857354,
 'anesthetic_-_local': 0.6159446890536782,
 'angiogenesis_inhibitor': 0.6073751647180867,
 'angiotensin_receptor_antagonist